### NIS-Seq, Figure 1E written by AH and PK

---

This script generates figure 1E in [*Cell Type-Agnostic Optical Perturbation Screening Using Nuclear In-Situ Sequencing (NIS-Seq)*](https://www.biorxiv.org/content/10.1101/2024.01.18.576210v1). The goal is to compare the percentage of mapped nuclei of *NIS-Seq* and [*Optical Pooled Screens*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6886477/). 


The inputs for this script were generated with [*JSB ImageFiend*](https://jsb-lab.bio/opticalscreening/AnalyzeInSituCombined_v1.htm) and consist of the follwing files for each celltype:

1. **ExperimentName_NuclearIntensities_normal.txt**
    * columns: tile number, nucleus id, thresholded intensity
        * *tile*: Tile number, according to order of files loaded; counting from 0
        * *nucleus*: Number of the nucleus in that tile, as defined by the masks loaded. Cellpose masks start counting at 1.
        * *intensity*: For each nucleus, the maximum intensity over all cycles and channels is indicated; intensity is calculated as the integrated pixel intensity across nuclear masks.
2. **ExperimentName_NuclearIntensities_scrambled.txt**
    * columns: tile number, nucleus id, thresholded intensity
        * *tile*: Tile number, according to order of files loaded; counting from 0
        * *nucleus*: Number of the nucleus in that tile, as defined by the masks loaded. Cellpose masks start counting at 1.
        * *intensity*: For each nucleus, the maximum intensity over all cycles and channels is indicated; intensity is calculated as the integrated pixel intensity across nuclear masks.
    
3. **ExperimentName_NuclearSequences.txt**
    * columns: tile number, nucleus id, x and y coordinates, barcode sequence, maximal intensity
        * *tile*: Tile number, according to order of files loaded; counting from 0
        * *nucleus*: Number of the nucleus in that tile, as defined by the masks loaded. Cellpose masks start counting at 1.
        * *x*: Pixel-wise center position of each nuclear mask in the first cycle.
        * *y*: Pixel-wise center position of each nuclear mask in the first cycle.
        * *sequence*: Library-matched consensus sequence detected in a nucleus.
        * *max_intensity*: For each nucleus, the maximum intensity over all cycles and channels is indicated; intensity is calculated as the integrated pixel intensity across nuclear 
    
4. **ExperimentName_CellAssignments.txt**
    * columns: in-situ_tile, situ_nucleus, in-situ, in-situ_nucleus_x, in-situ_nucleus_y, in-situ_nucleus_area, phenotype_tile, phenotype_cell, phenotype_nucleus_x, phenotype_nucleus_y, phenotype_nucleus_area
        * *in-situ_tile*: Tile number, according to order of files loaded; counting from 0
        * *in-situ_nucleus*: Number of the nucleus in that tile, as defined by the masks loaded. Cellpose masks start counting at 1
        * *in-situ_nucleus_x*: Pixel-wise center position of each nuclear mask in the first cycle
        * *in-situ_nucleus_y*: Pixel-wise center position of each nuclear mask in the first cycle
        * *in-situ_nucleus_area*: Pixel-wise area of each nuclear mask in the first cycle
        * *phenotype_tile*: Tile number, according to order of files loaded; counting from 0
        * *phenotype_cell*: Number of the membrane mask in that tile, as defined by the masks loaded. Cellpose masks start counting at 1
        * *phenotype_nucleus_x*: Pixel-wise center position of each nuclear mask
        * *phenotype_nucleus_y*: Pixel-wise center position of each nuclear mask
        * *phenotype_nucleus_area*: Pixel-wise area of each nuclear mask in the phenotype data
    
The optical pooled screening analysis pipeline can be installed from [*Here*](https://github.com/feldman4/OpticalPooledScreens_2019).

---

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

# Optical pooled screening libraries
from ops.imports import *
from ops.process import Align
import ops.firesnake
from ops.firesnake import Snake

# Plot generation
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
inp_nis = '/home/jsblab/nis_analysis/'
output = '/home/jsblab/nis_analysis_analysis/'

celltypes = set([x.split('_')[0] for x in os.listdir(inp_nis) if x[0] != '.'])

### NIS-Seq analysis

In [ ]:
dic = {}
mapping = {'standard': pd.DataFrame(), 'scrambled': pd.DataFrame()}

# Iterate through the celltypes
for celltype in celltypes:

    
    # Load Image Fiend data
    nucs = pd.read_csv(inp_nis + f'{celltype}_nuclear_intensities_normal.txt', sep = '\t')
    seqs = pd.read_csv(inp_nis + f'{celltype}_NuclearSequences_normal.txt', sep = '\t')
    
    # Scrambled data
    nucs_sc = pd.read_csv(inp_nis + f'{celltype}_nuclear_intensities_scrambled.txt', sep = '\t')
    seqs_sc = pd.read_csv(inp_nis + f'{celltype}_NuclearSequences_scrambled.txt', sep = '\t')
    
    # In-situ phenotype analysis
    assi = pd.read_csv(inp_nis + f'P42_{celltype}_CellAssignments.txt', sep = '\t')
    
    # merge sequences with assignments
    seqs = seqs.merge(assi, left_on=['tile','nucleus'], right_on=['in-situ_tile','cell'])
    tiles = np.unique(seqs.tile).tolist()
    mapping['standard'] = pd.concat([mapping['standard'], pd.DataFrame([(celltype, (nucs.tile == tile).sum(), (seqs.tile == tile).sum()) for tile in tiles])])
    
    # merge sequences with assignments
    seqs_sc = seqs_sc.merge(assi, left_on=['tile','nucleus'], right_on=['in-situ_tile','cell'])
    tiles = np.unique(seqs_sc.tile).tolist()
    mapping['scrambled'] = pd.concat([mapping['scrambled'], pd.DataFrame([(celltype, (nucs_sc.tile == tile).sum(), (seqs_sc.tile == tile).sum()) for tile in tiles])])


mapping['standard'] = mapping['standard'].rename({0: 'celltype', 1: 'nucs', 2: 'seqs'}, axis=1)
mapping['scrambled'] = mapping['scrambled'].rename({0: 'celltype', 1: 'nucs', 2: 'seqs'}, axis=1)

dic = {'standard': {}, 'scrambled': {}}

for x in ['standard', 'scrambled']:
    for celltype in celltypes:
        df_ = mapping[x].query(f'celltype=="{celltype}"')

        m1 = (df_.iloc[:(len(df_)//2)].seqs.sum() / df_.iloc[:(len(df_)//2)].nucs.sum() * 100).mean()
        m2 = (df_.iloc[(len(df_)//2):].seqs.sum() / df_.iloc[(len(df_)//2):].nucs.sum() * 100).mean()

        dic[x][celltype] = (m1, m2)

In [ ]:
df = pd.DataFrame(dic['standard']).T.rename({0: 'standard', 1: 'standard'}, axis=1)
df['0'] = pd.DataFrame(dic['scrambled']).T[0]
df['1'] = pd.DataFrame(dic['scrambled']).T[1]


df = df.rename({'0': 'scrambled', '1': 'scrambled'}, axis=1).rename(name_map)

In [ ]:
df.to_csv(output + f'NIS_results_duplicates.csv')

### Optical pooled screens analyis pipeline

This requires you to input all the image data for the phenotype and the in-situ cycles.

In [ ]:
def align_images(img1, img2, targetimage):
    images = img1, img2
    _, offset = ops.process.Align.calculate_offsets(images, upsample_factor=2)
    
    img_pad = np.pad(targetimage, mode = 'constant', pad_width = int(np.max(abs(offset)))+1, constant_values = 0)
    mx = int(img_pad.shape[0]/2 + offset[0])
    my = int(img_pad.shape[0]/2 + offset[1])
    return img_pad[mx-1024:mx+1024, my-1024:my+1024]

In [ ]:
def call_cells(array, cell, WILDCARDS, THRESHOLD_STD = 10):
    aligned = Snake._align_SBS(array, method='DAPI')
    loged = Snake._transform_log(aligned, sigma = 1, skip_index=0)
    maxed = Snake._max_filter(loged, 3, remove_index=0)
    std = Snake._compute_std(loged, remove_index=0)
    peaks = Snake._find_peaks(std)
    df_bases = Snake._extract_bases(maxed, peaks, cell, 
                        THRESHOLD_STD, wildcards=WILDCARDS)
    if len(df_bases)>0:
    
        df_reads = Snake._call_reads(df_bases)
        df_cells = Snake._call_cells(df_reads)
        return df_cells

In [ ]:
brunello = pd.read_csv('data/starting/brunello_library.txt', sep = '\t', header = None)
brunello[2] = [i[:14] for i in brunello[1]]

brunello_scbl = pd.read_csv('data/starting/brunello_library_scrambled.txt', sep = '\t', header = None)
brunello_scbl[2] = [i[:14] for i in brunello_scbl[1]]

In [ ]:
wells = ['E3', 'D10', 'D5', 'E2', 'F4', 'G6', 'H7']
output = 'data/figE/'

In [ ]:
%%capture
res = {'standard': {'G9': 0}, 'scrambled': {'G9': 0}}
       
for well in wells:
    
    base = f'/home/jsblab/data2/P41_in-situ/{well}/'
    
    files = [x for x in os.listdir(base) if not os.path.isdir(base+x)]
    
    cycles = sorted(list(set([i.split('_')[0] for i in files])), key = lambda x:int(x[5:]))
    wells = sorted(list(set([i.split('_')[1] for i in files])), key = lambda x:(x[0],int(x[1])))
    tiles = sorted(list(set([i.split('_')[3] for i in files])), key = lambda x:int(x[4:]))
    channels = sorted(list(set([i.split('_')[4][7:9] for i in files])), key = lambda x:int(x))

    mappings = {'standard':  pd.DataFrame(columns=['tile','mapping_cells', 'cells', 'ratio']),
                'scrambled': pd.DataFrame(columns=['tile','mapping_cells', 'cells', 'ratio'])}
    
    df_stand = pd.DataFrame()
    df_scrbl = pd.DataFrame()
    df_ = pd.DataFrame()
    
    for tile in tiles:
        
        # Open Cell mask and reset array
        cells = np.array(Image.open(base + f'channel04/cp_cycle14_{well}_time001_{tile}.tif'))
        arr = np.zeros((14,5,2048,2048))
        
        if np.max(cells) == 0:
            continue
        
        # Load images into array
        for cycle_no, cycle in enumerate(cycles):
            for channel_no, channel in enumerate(channels):
                arr[cycle_no][channel_no] = np.array(Image.open(base + f'{cycle}_{well}_time001_{tile}_channel{channel}.tif'))
        
        # Align cells from first and 14. cycle
        cells = align_images(arr[0][0], arr[13][0], cells)     
        
        df_ = call_cells(arr, cells, THRESHOLD_STD=100, WILDCARDS = dict(well=f'{well}', tile=f'{tile}'))
        dfs_ = call_cells(arr, cells, THRESHOLD_STD=10, WILDCARDS = dict(well=f'{well}', tile=f'{tile}'))
        
        # If no cells are detected in tile
        if df_ is None and dfs_ is None:
            mappings['standard'].loc[len(mappings['standard'])]   = [tile, 0, np.max(cells), 0]
            mappings['scrambled'].loc[len(mappings['scrambled'])] = [tile, 0, np.max(cells), 0]
            continue
            
        # Concat different thresholds
        dfc = pd.concat([df_, dfs_])
        dfc = dfc.drop_duplicates(['tile','well','cell'])

        # Merge with brunello library
        df_stand = df_stand.append(pd.concat([pd.merge(dfc, brunello, left_on='cell_barcode_0', right_on=[2]),
                         pd.merge(dfc, brunello, left_on='cell_barcode_1', right_on=[2])]).drop_duplicates(['tile', 'cell']))

        df_scrbl = df_scrbl.append(pd.concat([pd.merge(dfc, brunello_scbl, left_on='cell_barcode_0', right_on=[2]),
                          pd.merge(dfc, brunello_scbl, left_on='cell_barcode_1', right_on=[2])]).drop_duplicates(['tile', 'cell']))
        
        
        
        mappings['standard'].loc[len(mappings['standard'])]   = [tile, len(df_stand[df_stand.tile == tile]), np.max(cells), len(df_stand[df_stand.tile == tile])/np.max(cells)*100]
        mappings['scrambled'].loc[len(mappings['scrambled'])] = [tile, len(df_scrbl[df_scrbl.tile == tile]), np.max(cells), len(df_scrbl[df_scrbl.tile == tile])/np.max(cells)*100]
        
    # Remove tiles with low amount of cells
    for x in ['standard', 'scrambled']:
        mappings[x] = mappings[x][mappings[x]['mapping_cells'] >= (np.percentile(mappings[x]['mapping_cells'], 50)/10)]

    mappings['standard'].to_csv(output + f'insitu_{well}_mapping.csv')
    mappings['scrambled'].to_csv(output + f'insitu_{well}_mapping_scrbl.csv')
    df_stand.to_csv(output + f'insitu_{well}_all_cells.csv')
    df_scrbl.to_csv(output + f'insitu_{well}_all_cells_scrbl.csv')
        
    # Merge with cell assignment data
    assignment = pd.read_csv(f'data/starting/figE/insitu/P41_{well}_CellAssignments.txt', sep = '\t')
    
    df_stand['tile'] = df_stand.tile.str[4:].astype(int)-1
    df_scrbl['tile'] = df_scrbl.tile.str[4:].astype(int)-1

    df_stand = assignment.merge(df_stand, left_on = ['in-situ_tile','cell'], right_on = ['tile','cell'])
    df_scrbl = assignment.merge(df_scrbl, left_on = ['in-situ_tile','cell'], right_on = ['tile','cell'])

    # Calculate mean in first halve and second
    for x in ['standard', 'scrambled']:

        sum_cells_front = sum([i for i in mappings[x].cells][:len(mappings[x])//2])
        sum_mapped_front = sum([i for i in mappings[x]['mapping_cells']][:len(mappings[x])//2])
        sum_cells_back = sum([i for i in mappings[x].cells][len(mappings[x])//2:])
        sum_mapped_back = sum([i for i in mappings[x]['mapping_cells']][len(mappings[x])//2:])

        res[x][well] = [sum_mapped_front/sum_cells_front*100, sum_mapped_back/sum_cells_back*100]        

In [ ]:
name_map = {'E2': 'MC-38', 'E3': 'MaMeI65', 'F4': 'B16-F1', 'D5': 'PC-3', 'G6': 'THP1', 'H7': 'iMac', 'D10': 'HeLa', 'G9': 'Primary'}

df = pd.DataFrame(res['standard']).T.rename({0: 'standard', 1: 'standard'}, axis=1)
df['0'] = pd.DataFrame(res['scrambled']).T[0]
df['1'] = pd.DataFrame(res['scrambled']).T[1]

df = df.rename({'0': 'scrambled', '1': 'scrambled'}, axis=1).rename(name_map)

In [ ]:
df.to_csv(output + f'final_insitu_merged.csv')

### Generate final figure

In [ ]:
# Load insitu and nis data and reformat columns
df_insitu = pd.read_csv(output + 'insitu_results_duplicates.csv', index_col=0).loc[['HeLa', 'MaMeI65', 'B16-F1', 'THP1', 'PC-3', 'iMac', 'MC-38', 'Primary']]
df_nis = pd.read_csv(output + 'NIS_results_duplicates.csv', index_col=0).loc[['HeLa', 'MaMeI65', 'B16-F1', 'THP1', 'PC-3', 'iMac', 'MC-38', 'Primary']]

# Reformat tables
resdf = {'cell': [], 'method': [], 'perc': []}

for df, name in zip([df_nis, df_insitu], ['NIS', 'insitu']):
    for i, row in df.iterrows():
        for col in df.columns:
            resdf['cell'].append(i)
            resdf['method'].append(f'{name}_scrambled' if 'scrambled' in col else name)
            resdf['perc'].append(row[col])
        
df = pd.DataFrame(resdf)

In [ ]:
# Final Figure E
g = sns.catplot(data=df, kind="bar", x="cell", y="perc", hue="method", palette="dark", alpha=.6, height=7, aspect=15/10)
g.set_axis_labels('', 'Nuclei with library barcode (%)')
g.set_titles('Figure E')
g.set(ylim=(0, 80))

plt.savefig(f'data/figE/FigureE_AH.svg')
plt.show()